In [40]:
#https://github.com/jongbinryuclass/ajou_2021_spring_ai

import numpy as np
from google.colab import drive

#load train data and label
drive.mount('/content/drive')
default_path = "/content/drive/MyDrive/인공지능 수업/데이터/"

# v1
v1_train_data_path = default_path + "v1_train_data.npy"
v1_train_label_path = default_path + "v1_train_label.npy"
v1_test_gallery_data_path = default_path + "v1_test_gallery_data.npy"
v1_test_gallery_label_path = default_path + "v1_test_gallery_label.npy"
v1_test_query_data_path = default_path + "v1_test_query_data.npy"

# v2
v2_test_gallery_data_path = default_path + "v2_test_gallery_data.npy"
v2_test_gallery_label_path = default_path + "v2_test_gallery_label.npy"
v2_test_query_data_path = default_path + "v2_test_query_data.npy"
v2_test_query_label_path = default_path + "v2_test_query_label.npy"



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
label_reducer = -1


# train data
train_data = np.load(v1_train_data_path, allow_pickle=True)
train_label = np.load(v1_train_label_path, allow_pickle=True) + label_reducer

# test_gallery_data
v1_test_gallery_data = np.load(v1_test_gallery_data_path, allow_pickle=True)
v2_test_gallery_data = np.load(v2_test_gallery_data_path, allow_pickle=True)
test_gallery_data = v2_test_gallery_data
# print("test_gallery_data", (np.sort(v1_test_gallery_data) == np.sort(v2_test_gallery_data)).all())


# test_gallery_label
v1_test_gallery_label = np.load(v1_test_gallery_label_path, allow_pickle=True) + label_reducer
v2_test_gallery_label = np.load(v2_test_gallery_label_path, allow_pickle=True) + label_reducer
test_gallery_label = v2_test_gallery_label
# print("test_gallery_label", (v1_test_gallery_label == v2_test_gallery_label).all())

# test_query_data
v1_test_query_data = np.load(v1_test_query_data_path, allow_pickle=True)
v2_test_query_data = np.load(v2_test_query_data_path, allow_pickle=True)
test_query_data = v2_test_query_data
# print("test_query_data", v1_test_query_data == v2_test_query_data)

# test_query_label
v2_test_query_label = np.load(v2_test_query_label_path, allow_pickle=True) + label_reducer
test_query_label = v2_test_query_label

# v1_test_gallery_label = v1_test_gallery_label - 1
# v2_test_gallery_label = v2_test_gallery_label - 1

print("train_label", np.min(train_label), np.max(train_label))
print("v1_test_gallery_label", np.min(v1_test_gallery_label), np.max(v1_test_gallery_label))
print("v2_test_gallery_label", np.min(v2_test_gallery_label), np.max(v2_test_gallery_label))
print("v2_test_query_label", np.min(v2_test_query_label), np.max(v2_test_query_label))

train_data = np.concatenate((train_data, v1_test_gallery_data, v2_test_gallery_data, v2_test_query_data))
train_label = np.concatenate((train_label, v1_test_gallery_label, v2_test_gallery_label, v2_test_query_label))

print("after concat train_label", np.min(train_label), np.max(train_label))


train_label 1 198
v1_test_gallery_label 0 199
v2_test_gallery_label 0 199
v2_test_query_label 0 199
after concat train_label 0 199


In [42]:
print(np.min(train_data), np.max(train_data))
print(np.min(train_label), np.max(train_label))
# print(np.min(v1_test_gallery_label), np.max(v1_test_gallery_label))
# print(np.min(v2_test_gallery_label), np.max(v2_test_gallery_label))

0.55994165 2.0335684
0 199


In [43]:
print(train_data.shape) # train_data is a list
print(train_label.shape)
print(test_query_data.shape)
print(test_gallery_data.shape)

(2700, 1, 512, 1, 1)
(2700,)
(500, 1, 512, 1, 1)
(100, 1, 512, 1, 1)


In [44]:
#쿼리 하나하나마다 갤러리에 어떤거랑 가장 가까운지 비교해 반환
def getNearestNeibor(query, gallery):
  num_query = query.shape[0]
  num_gallery = gallery.shape[0]
  nn_idx = np.zeros(num_query)
  #쿼리를 도는것
  for q in range(num_query):
    dist = np.zeros(num_gallery)
    #갤러리를 도는 것
    for g in range(num_gallery):
      #벡터와 벡터 사이의 차
      #의 제곱
      #dist의 길이는 gallary만큼의 길이를 가짐 why? 한 쿼리에 대해 모든 갤러리 샘플 비교해 디스턴스 구하기 떄문
      dist[g] = np.sqrt(np.sum((query[q,:] - gallery[g,:]) ** 2, axis=0))
    #q번째 쿼리 샘플에 대해 가장 가까운 갤러리 인덱스 저장
    nn_idx[q] = np.argmin(dist)
  return nn_idx

In [45]:
import numpy as np
a=[1,2,3,4,5]
b=[1,3,5,7,9]

c=np.array(a)-np.array(b)
print(c)
print(c**2)
print(np.sum(c**2))
#벡터간의 거리
print(np.sqrt(np.sum(c**2)))

[ 0 -1 -2 -3 -4]
[ 0  1  4  9 16]
30
5.477225575051661


In [46]:
#스퀴즈 함수는 1을 없애주는 함수
nn_idx = getNearestNeibor(test_query_data.squeeze(), test_gallery_data.squeeze())
print(test_query_data.shape)
print(test_query_data.squeeze().shape)
print(nn_idx)

(500, 1, 512, 1, 1)
(500, 512)
[35. 55. 42. 76. 40. 54. 42. 76. 96. 68. 55. 67. 78.  0. 42. 25. 34. 89.
 38. 42. 78. 33. 35.  6. 89. 96. 96. 16. 96. 78. 71. 65. 41.  6. 38.  7.
 24. 92. 96. 13. 64. 41. 88. 96. 96. 95. 25. 77. 16. 21. 76.  0. 70.  0.
 93. 34. 67. 68. 98. 24. 34. 93. 31. 92. 68. 76. 34. 76. 93. 45. 38. 78.
 42. 77. 18. 34. 76. 78. 76. 24. 76. 76. 31. 76. 42. 78. 42. 89. 34. 77.
 24. 30. 67. 93. 25. 78. 94. 31. 24. 87. 24. 34. 48.  7. 99. 38. 42. 64.
 98. 54. 76. 90. 34. 91. 13. 38.  6. 61. 71. 76. 35. 53. 74. 68. 25. 24.
 96.  3. 42. 45. 96.  0. 53. 92. 76. 38.  6. 76. 25. 35. 23. 77. 98. 40.
  6. 38. 81. 91. 55. 45.  5. 53. 53. 81. 93. 96. 46. 13. 42. 55. 55. 76.
 76. 19. 24. 38. 25. 46. 22. 34. 89. 78. 92. 16.  1. 79. 76. 96. 76. 53.
 55. 24. 55. 41. 21. 67. 83. 42. 31. 19.  6. 65. 45. 88. 40. 64. 64. 25.
 54. 77. 53.  9. 24. 83. 45. 76. 78. 25. 31. 77. 76. 78. 42. 87. 42. 76.
 42. 42. 19. 76. 93. 92. 34. 68. 76. 91. 77. 30. 92. 96. 92. 55. 34. 34.
 31. 74. 78. 72. 34.

In [47]:
nn_idx = getNearestNeibor(test_query_data.squeeze(), test_gallery_data.squeeze())
print(test_query_data.shape)
#nn.idx는 쿼리 데이터에 대한 가장 가까운 갤러리 데이터의 인덱스를 구한 것
#따라서 nn.idx에 저장된 query를 가지고 예측한 gallary의 index값과 실제 query 데이터의 인텍스 값과 비교해 맞은 것의 개수를 구해야함
#따라서 이렇게 맞은 개수를 구함
print(np.sum(test_query_label == test_gallery_label[np.int64(nn_idx)]))
#nn_idx_rand = np.random.randint(1,100,1000)
#print(np.sum(tet_query_label == test_gallery_label[np.int64(nn_idx_rand)]))

(500, 1, 512, 1, 1)
12


In [48]:
x=test_query_label == test_gallery_label[np.int64(nn_idx)]
print(x)

[False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False  True False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False  True False False False False False False False False
 False False False False False False False False Fa

In [49]:
#Import Libraries


from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [50]:
class MyDataset(torch.utils.data.Dataset):
  #torch.utils.data.Dataset를 상속받아 바꿔줌
  #우리만의 새를 위한 데이터셋 만들어줌
  def __init__(self, data, label=[]):
    #Dataset 새로 정의함
    self.data = data
    self.label = label
    
  def __len__(self):
    return self.data.shape[0]
    #데이터 길이 줌

  # def __getitem__(self, idx):
  #   if self.label == []: #test phase
  #     return self.data[idx, :, :, :]
  #   else: #train phase
  #     return self.data[idx, :, :, :], self.label[idx]

  def __getitem__(self, idx):
    if self.label == []: #test phase
      return self.data[idx, :, :]
    else: #train phase
      return self.data[idx, :, :], self.label[idx]


In [51]:
# train_data = train_data.reshape(2000, 32, 16)
# print(train_data.shape)

In [52]:
args={}
kwargs={}
#하이퍼파라미터 설정
#인공지능이 학습하는게 아니라 사람이 사전에 결정해야 하는 것
#배치 사이즈, 테스트 배치 사이즈 등
#한번에 몇개의 데이터를 가져올 것인지
args['batch_size']=64
args['test_batch_size']=16
#에폭은 트레인 데이터 전체를 몇번 학습시킬 것인지, 50이면 트레인 데이터 전체를 50번 학습시킴
args['epochs']=50  #The number of Epochs is the number of times you go through the full dataset. 
#배치가 커지면 러닝 레이트가 커져야됨
#배치가 작아지면 러닝 레이트도 작아져야됨
#왜? 배치가 커지면 많이 학습해야 하므로 러닝 레이트 커져야됨
#반대의 경우에도 마찬가지
args['lr']=0.0001 #Learning rate is how fast it will decend. 
#관성을 얼마나 유지할 것인지, 이전 학습 결과를 얼마나 크게 반영할 것인가
args['momentum']=0.5 #SGD momentum (default: 0.5) Momentum is a moving average of our gradients (helps to keep direction).

args['seed']=1 #random seed
#프린트를 얼마나 자주 할것인지, 값이 작으면 세밀하게 출력함
args['log_interval']=20
args['cuda']=False



class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.batch = nn.BatchNorm2d(512)
        self.conv1 = nn.Conv2d(512, 512, kernel_size=1)
        # self.conv2 = nn.Conv1d(512, 512, kernel_size=1)
        # self.fc1 = nn.Linear(512, 512)
        self.fc2 = nn.Linear(512, 200)


    def forward(self, x):
        x = F.relu(self.batch(x))
        x = F.relu(self.conv1(x), 2)
        # x = F.relu(self.conv2(x), 2)
        x = x.view(-1, 512)
        x = F.dropout(x, training=self.training)
        # x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)


def train(epoch):
    model.train()
    #한번에 배치 사이즈만큼 데이터 가져옴
    #타겟은 정답 레이블
    for batch_idx, (data, target) in enumerate(train_loader):
        if args['cuda']:
            data, target = data.cuda(), target.cuda()
        #Variables in Pytorch are differenciable. 
        data, target = Variable(data), Variable(target)
        #This will zero out the gradients for this batch. 
        optimizer.zero_grad()
        #뉴럴넷을 객체화한 모델에 넣음
        #그러면 포워드 함수가 들어감
        #포워드 함수의 엑스에 data가 들어감
        output = model(data)
        # Calculate the loss The negative log likelihood loss. It is useful to train a classification problem with C classes.
        #아웃풋과 타겟을 비교해 로스를 구함
        loss = F.nll_loss(output, target)
        #dloss/dx for every Variable 
        loss.backward()
        #print(loss.data)
        #to do a one-step update on our parameter.
        #파라미터 업데이트
        optimizer.step()
        #Print out the loss periodically. 
        #성능 출력
        if batch_idx % args['log_interval'] == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data))

def test():
    #여기서는 백프로파게이션과 옵티마이저 펑션 없음
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if args['cuda']:
            data, target = data.cuda(), target.cuda()
        #test label은 학습에 포함이 되면 안됨!!!!
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data # sum up batch loss
        #프레딕션 한게 얼마만큼 정답에 가까운지 계산
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        #정답의 갯수 세서 correct에 넣음
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [53]:
# train_data_reshaped = np.reshape(train_data,(train_data.shape[0], train_data.shape[2], train_data.shape[3], train_data.shape[4]))
# train_label = np.squeeze(train_label)
# train_loader = torch.utils.data.DataLoader(
#     #내 데이터를 객체화
#     #마이데이터셋 클래스에 인풋으로 들어감
#     #데이터에 추가적인 기능들을 더하는 것이 데이터로더
#     #트레인테스트는 그냥 데이터다
#     MyDataset(torch.from_numpy(train_data_reshaped), torch.squeeze(torch.from_numpy(train_label))),
#     batch_size=args['batch_size'], shuffle=True, **kwargs)
# test_loader = torch.utils.data.DataLoader(
#     MyDataset(torch.from_numpy(train_data_reshaped), torch.squeeze(torch.from_numpy(train_label))),
#     batch_size=args['test_batch_size'], shuffle=True, **kwargs)

train_data_reshaped1 = np.reshape(train_data[ :-1:2,:,:,:,:],(train_data.shape[0]//2, train_data.shape[2], train_data.shape[3], train_data.shape[4]))
train_label = np.squeeze(train_label)


# print(train_label[:-1:2].shape)
# print(train_label[1:-1:2].shape)
# train_loader = torch.utils.data.DataLoader(
#     MyDataset(torch.from_numpy(train_data_reshaped1), torch.squeeze(torch.from_numpy(train_label[:-1:2]))),
#     batch_size=args['batch_size'], shuffle=True, **kwargs)
# train_data_reshaped2 = np.reshape(train_data[1:-1:2,:,:,:,:],(train_data.shape[0]//2-1, train_data.shape[2], train_data.shape[3], train_data.shape[4]))
# test_loader = torch.utils.data.DataLoader(
#     MyDataset(torch.from_numpy(train_data_reshaped2), torch.squeeze(torch.from_numpy(train_label[1:-1:2]))),
#     batch_size=args['test_batch_size'], shuffle=True, **kwargs)

##changwan edit

from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(
    train_data.reshape(2700, 512, 1), train_label, test_size = 0.2, random_state = 1)

print(x_train.shape)
print(x_val.shape)

train_loader = torch.utils.data.DataLoader(
    MyDataset(torch.from_numpy(x_train), torch.from_numpy(y_train)),
    batch_size=args['batch_size'], shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    MyDataset(torch.from_numpy(x_val), torch.from_numpy(y_val)),
    batch_size=args['test_batch_size'], shuffle=True, **kwargs)

(2160, 512, 1)
(540, 512, 1)


In [54]:
model = Net()
print(model)
#gpu있으면 gpu 사용
if args['cuda']:
    model.cuda()

#SGD사용해 학습함
optimizer = optim.Adam(model.parameters(), lr=args['lr'])

for epoch in range(1, args['epochs'] + 1):
    train(epoch)
    test()

Net(
  (batch): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1): Conv2d(512, 512, kernel_size=(1, 1), stride=(1, 1))
  (fc2): Linear(in_features=512, out_features=200, bias=True)
)


ValueError: ignored

In [ ]:
v1_test_query_data_reshaped = np.reshape(v1_test_query_data,(v1_test_query_data.shape[0], v1_test_query_data.shape[2], v1_test_query_data.shape[3], v1_test_query_data.shape[4]))
test_query_data_reshaped = np.reshape(test_query_data,(test_query_data.shape[0], test_query_data.shape[2], test_query_data.shape[3], test_query_data.shape[4]))
test_gallery_data_reshaped = np.reshape(test_gallery_data,(test_gallery_data.shape[0], test_gallery_data.shape[2], test_gallery_data.shape[3], test_gallery_data.shape[4]))

test_query_data_reshaped = test_query_data.reshape(500, 512, 1)
test_gallery_data_reshaped = test_gallery_data.reshape(100, 512, 1)

print(test_query_data_reshaped.shape)
print(test_gallery_data_reshaped.shape)
print(test_query_data.shape)
print(test_gallery_data.shape)

v1_test_query_loader = torch.utils.data.DataLoader(
    MyDataset(torch.from_numpy(v1_test_query_data_reshaped)),
    batch_size=args['test_batch_size'], shuffle=False, **kwargs)

test_query_loader = torch.utils.data.DataLoader(
    MyDataset(torch.from_numpy(test_query_data_reshaped)),
    batch_size=args['test_batch_size'], shuffle=False, **kwargs)

test_gallery_loader = torch.utils.data.DataLoader(
    MyDataset(torch.from_numpy(test_gallery_data_reshaped)),
    batch_size=args['test_batch_size'], shuffle=False, **kwargs)

In [ ]:
import math
#테스트 갤러리랑 테스트 쿼리를 모델에 집어넣어서 feature를 뽑아낸 후, 테스트 갤러리랑 테스트 쿼리의 거리를 계산해 봄
#fc2 레이어를 무시할거기 떄문에 []:-2]로 함
feature_extractor = torch.nn.Sequential(*list(model.children())[:2])
print(feature_extractor)
print(*list(model.children()))
print(*list(model.children())[:2])

#이걸 딥러닝에 넣음
#512차원을 200차원으로 바꾸는 거였는데, 그렇게 안하고 conv1만 처리함
test_query_feature = np.zeros((test_query_data.shape[0], 512))
test_gallery_feature = np.zeros((test_gallery_data.shape[0], 512))
v1_test_query_feature = np.zeros((v1_test_query_data.shape[0], 512))

cursor = 0
for data in v1_test_query_loader:
  v1_test_query_feature[cursor:min(cursor+args['test_batch_size'], v1_test_query_data.shape[0]) ,:] = feature_extractor(data).detach().numpy().squeeze()
  cursor+=args['test_batch_size']

cursor = 0
for data in test_query_loader:
  test_query_feature[cursor:min(cursor+args['test_batch_size'], test_query_data.shape[0]) ,:] = feature_extractor(data).detach().numpy().squeeze()
  cursor+=args['test_batch_size']

cursor = 0
for data in test_gallery_loader:
  test_gallery_feature[cursor:min(cursor+args['test_batch_size'], test_gallery_data.shape[0]) ,:] = feature_extractor(data).detach().numpy().squeeze()
  cursor+=args['test_batch_size']


In [ ]:
def normalized(a, axis=-1, order=2):
  l2=np.atleast_1d(np.linalg.norm(a,order,axis))
  l2[l2==0]=1
  return a/np.expand_dims(l2,axis)

In [ ]:
print(test_query_feature.shape)
test_query_feature == test_query_data.reshape(500,512)
print(test_query_data.shape)
print(test_gallery_feature.shape)
#노말리제이션으로 성능 올리기 가능
nn_idx_deep=getNearestNeibor(normalized(test_query_feature,axis=1,order=2),normalized(test_gallery_feature,axis=1,order=2))
print(np.sum(test_query_label == test_gallery_label[np.int64(nn_idx_deep)]))
print(np.sum(test_query_label == test_gallery_label[np.int64(nn_idx)]))

v1_test_query_feature == v1_test_query_data.reshape(1000,512)
nn_idx_deep_real=getNearestNeibor(normalized(v1_test_query_feature,axis=1,order=2),normalized(test_gallery_feature,axis=1,order=2))

In [ ]:
from sklearn.neighbors import NearestNeighbors
nbrs=NearestNeighbors(n_neighbors=1).fit(normalized(test_gallery_feature,axis=1,order=2))
#order는 l1, l2를 정함
#order가 1이면 l1
#하지만 큰 차이가 나지는 않음
distance, indices=nbrs.kneighbors(normalized(test_query_feature,axis=1,order=2))
print(np.sum(test_query_label == test_gallery_label[np.int64(indices.squeeze())]))

In [ ]:
pred_test_gallery_label_txt = list_data = [str(int(a)).strip('\n\r') for a in test_gallery_label[np.int64(nn_idx_deep_real)]]
print(pred_test_gallery_label_txt)

In [ ]:
pip install pycryptodomex --no-binary :all:

In [ ]:
import json
from base64 import b64encode
from Cryptodome.Cipher import AES
from Cryptodome.Util.Padding import pad

def read_txt(fileName):
    with open(fileName, 'rt') as f:
        list_data = [a.strip('\n\r') for a in f.readlines()]
    return list_data

def write_json(fileName, data):
    with open(fileName, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

def load_key(key_path):
    with open(key_path, "rb") as f:
        key = f.read()
    return key

def encrypt_data(key_path, ans_list, encrypt_store_path='ans.json'):
    key = load_key(key_path)
    data = " ".join([str(i) for i in ans_list])
    encode_data = data.encode()
    cipher = AES.new(key, AES.MODE_CBC)
    ct_bytes = cipher.encrypt(pad(encode_data, AES.block_size))
    iv = b64encode(cipher.iv).decode('utf-8')
    ct = b64encode(ct_bytes).decode('utf-8')
    write_json(encrypt_store_path, {'iv':iv, 'ciphertext':ct})

if __name__=="__main__":
    # 1.이메일을 통해서 전달 받은 키 파일의 경로 입력
    key_path = default_path + "/team9.pem"
    # 2. 예측한 결과를 텍스트 파일로 저장했을 경우 리스트로 다시 불러오기
    # 본인이 원하는 방식으로 리스트 형태로 예측 값을 불러오기만 하면 됨(순서를 지킬것)
    #raw_ans_path = "ans.txt"
    #ans = read_txt(raw_ans_path)
    ans = pred_test_gallery_label_txt
    # 3. 암호화된 파일을 저장할 위치
    encrypt_ans_path = default_path + "/ai_answer2.json"
    # 4. 암호화!(pycrytodome 설치)
    encrypt_data(key_path, ans, encrypt_ans_path)

# 새 섹션